In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext,SparkConf
spark = SparkSession.builder.appName('classification_task').getOrCreate()

In [4]:
df=spark.read.csv(r'./zzd.csv',header=True,inferSchema=True)

In [5]:
df.show(2)

+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+
|   1|  2|     3|   4|   5|     6|          7|          8|          9|         10|         11|         12|   13| 14|
+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+
|7.72| 37|124.26|3.66|3.41|0.2536|27.43926829|      0.435|1.575714286|60.32738095|6.131579483|15.82402098|85.99|  1|
|9.17| 77|121.36| 7.8| 4.1|4.7898|29.81151943|0.373038869|1.874487633|170.0271378|2.046075682| 21.8880885|96.16|  1|
+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+
only showing top 2 rows



In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol="14", outputCol="lable")
si_model = stringIndexer.fit(df)
td = si_model.transform(df)

In [8]:
td.show(2)

+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+-----+
|   1|  2|     3|   4|   5|     6|          7|          8|          9|         10|         11|         12|   13| 14|lable|
+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+-----+
|7.72| 37|124.26|3.66|3.41|0.2536|27.43926829|      0.435|1.575714286|60.32738095|6.131579483|15.82402098|85.99|  1|  1.0|
|9.17| 77|121.36| 7.8| 4.1|4.7898|29.81151943|0.373038869|1.874487633|170.0271378|2.046075682| 21.8880885|96.16|  1|  1.0|
+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+-----+
only showing top 2 rows



In [10]:
from pyspark.ml.feature import VectorAssembler 
assembler = VectorAssembler(inputCols = ['1','2','3','4','5'], outputCol = 'features')
final_df = assembler.transform(td)
final_df.show(2)

+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+-----+--------------------+
|   1|  2|     3|   4|   5|     6|          7|          8|          9|         10|         11|         12|   13| 14|lable|            features|
+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+-----+--------------------+
|7.72| 37|124.26|3.66|3.41|0.2536|27.43926829|      0.435|1.575714286|60.32738095|6.131579483|15.82402098|85.99|  1|  1.0|[7.72,37.0,124.26...|
|9.17| 77|121.36| 7.8| 4.1|4.7898|29.81151943|0.373038869|1.874487633|170.0271378|2.046075682| 21.8880885|96.16|  1|  1.0|[9.17,77.0,121.36...|
+----+---+------+----+----+------+-----------+-----------+-----------+-----------+-----------+-----------+-----+---+-----+--------------------+
only showing top 2 rows



In [12]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="lable", seed=42)
model = rf.fit(final_df)
model.featureImportances

SparseVector(5, {0: 0.4705, 1: 0.1962, 2: 0.3333})

In [13]:
model.explainParams()

"cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: False)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)\nfeatureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)\nfeaturesCol: features column name (default: features)\nimpurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)\nlabelCol: label column name (default: label, current: lable)\nmaxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number o

In [16]:
test0 = spark.createDataFrame([(Vectors.dense(23.0, 32,23.1,43,312),)], ["features"])
test0.show(2)

+--------------------+
|            features|
+--------------------+
|[23.0,32.0,23.1,4...|
+--------------------+



In [19]:
result = model.transform(test0).head()
result.prediction

0.0

In [20]:
result.probability

DenseVector([0.7132, 0.2868])